This tutorial walks you through using the HaloAnalysis package to read and use halo catalogs generated by Rockstar and merger trees generated via ConsistentTrees, from Gizmo simulations.

@author: Andrew Wetzel <arwetzel@gmail.com>

First, move within a simulation directory that contains a sub-directory 'halo/' that in turn contains a directory named rockstar/ or rockstar_dm/. (By convention, rockstar/ implies that Rockstar halo finding ran using all particle species, dark matter + stars + gas, while rockstar_dm/ implies that Rockstar halo finding ran using only dark-matter particles, which can produce more stable behavior.) By default, this HaloAnalysis package assumes that the raw text files that Rockstar produces have been converted to hdf5 format files that are in halo/rockstar_dm/catalog_hdf5/.

Currently we run Rockstar halo finder using only dark-matter particles. This means that all halo quantities are computed using **only** dark-matter particles. Within halo/rockstar_dm/catalog_hdf5/, halo_*.hdf5 (one file per snapshot) and tree.hdf5 (one file across all snapshots) thus contain halo information based only on dark-matter particles.

HaloAnalysis can assign baryonic (star or gas) particles to dark-matter halos in post-processing. By default, HaloAnalysis stores these properties for stars in files named catalog_hdf5/star_*.hdf5, with 1 file per snapshot. By default, the HaloAnalysis reader looks for these star files, and if they exist, it assigns those properties to the halo catalogs or trees when you read them. You can disable this, to read only dark-matter halo properties for speed/efficiency, by setting species=None in read_catalogs() or read_tree().

Ensure that the halo_analysis and utilities directories are in your python path, then...

In [2]:
import halo_analysis as halo
import utilities as ut

import numpy as np

In [ ]:
# you can access the individual files/modules as named or use the aliases in __init__.py for convenience/brevity. for example, these are the same:

halo.halo_io
halo.io

# read halo catalog

In [ ]:
# we recommend that you copy this jupyter notebook tutorial into a simulation directory 
# (for example, m12i_res7100/) and run from there.
# however, you can set simulation_directory below to point to any simulation directory and then run this notebook from anywhere

# use this is you are running from within a simulation directory
#simulation_directory = '.'

# use this to point to a specific simulation directory, if you run this notebook from somwhere else
simulation_directory = '/Users/awetzel/work/research/simulation/gizmo/simulations/m12/m12i/m12i_res7100'

In [ ]:
# read a halo catalog at single snapshot (z = 0)

hal = halo.io.IO.read_catalogs('redshift', 0, simulation_directory)

In [ ]:
# hal is a dictionary of halo properties

for k in hal.keys():
    print(k)

In [ ]:
# read halo catalogs at all available snapshots by supplying None or 'all' as the input snapshot value.
# read_catalogs() returns this as a list of dictionaries, with the list index being the snapshot index.
# beware - this can take a while to read...

hals = halo.io.IO.read_catalogs('redshift', None, simulation_directory)

In [ ]:
# halo catalog at z = 0 (snapshot 600)
hal_z0 = hals[600]

# halo catalog at z = 1 (snapshot 277)
hal_z1 = hals[277]

In [ ]:
# else, you can read just a few specific snapshots
# by default, it stores them in a (mostly emply) list, so the halo catalog list index = snapshot index

hals = halo.io.IO.read_catalogs('redshift', [0, 1], simulation_directory)

# halo catalog at z = 0 (snapshot 600)
hal_z0 = hals[600]

# halo catalog at z = 1 (snapshot 277)
hal_z1 = hals[277]

In [ ]:
# alternately, if you want a compact list (so halo catalog list index != snapshot index)

hals = halo.io.IO.read_catalogs('redshift', [0, 1], simulation_directory, all_snapshot_list=False)

hal_z1 = hals[0]
hal_z0 = hals[1]

# halo properties

In [ ]:
# read halo catalog at z = 0

hal = halo.io.IO.read_catalogs('redshift', 0, simulation_directory)

In [ ]:
# hal is a dictionary of halo properties

for k in hal.keys():
    print(k)

In [ ]:
# 3-D position (particle_number x dimension_number array) [kpc comoving]

hal['position']

In [ ]:
# 3-D velocity (particle_number x dimension_number array) [km/s physical]

hal['velocity']

In [ ]:
# DM mass of halo [M_sun]
# by default, I run the halo finder using 200m (200 x the mean matter density) for the default overdensity/virial definition

hal['mass']

In [ ]:
# but Rockstar also stores halo DM mass based on different virial definitions [M_sun]

print('{}\n{}\n{}'.format(hal['mass.200m'], hal['mass.vir'], hal['mass.200c']))

In [ ]:
# DM mass that is bound to halo [M_sun]

hal['mass.bound']

In [ ]:
# halo radius [kpc physical] again using 200m for the overdensity/virial definition 

hal['radius']

In [ ]:
# NFW scale radius [kpc physical]

hal['scale.radius']

In [ ]:
# maximum of the circular velocity profile [km/s physical]

hal['vel.circ.max']

In [ ]:
# standard deviation of the velocity (velocity dispersion) [km/s physical]

hal['vel.std']

In [ ]:
# the fraction of DM mass within R_200m that is low-resolution DM
# this is a derived quantity, so you need to call via the .prop() function (see below)

hal.prop('lowres.mass.frac')

In [ ]:
# index of the primary (most massive) host halo in the catalog

hal['host.index']

In [ ]:
# 3-D distance to the primary host halo [kpc physical]

hal['host.distance']

In [ ]:
# total (scalar) distance to the primary host halo [kpc physical]
# this is a derived quantity, so you need to call via the .prop() function (see below)

hal.prop('host.distance.total')

In [ ]:
# 3-D velocity wrt the primary host halo [kpc physical]
# radial and tangential velocity wrt the primary host halo [kpc physical]

print(hal['host.velocity'])
print(hal['host.velocity.rad'])
print(hal['host.velocity.tan'])

In [ ]:
# use .prop() to compute derived quantities
# this can handle simple arithmetic conversions, such as the log of the mass, or the ratio of masses
# see halo.io.HaloDictionaryClass for all options for derived quantities

print(hal.prop('host.distance.total'))
print(hal.prop('host.velocity.total'))
print(hal.prop('log mass'))
print(hal.prop('mass.bound / mass'))

# halo catalogs with baryonic particle properties

HaloAnalysis can assign baryonic (star or gas) particle properties to dark-matter halos in post-processing after generating the dark-matter halo catalogs. This package stores these baryonic properties in separate files, such as star_600.hdf5 for star particles at snapshot 600.

By default, the HaloAnalysis reader automatically looks if star files exist at a snapshot that you read, and it will read and append these star properties to the halo catalog. You can disable this if you want to read only dark-matter halo properties for speed/efficiency, by setting species=None in read_catalogs().

In [ ]:
# explicit input to ensure that it reads star particle properties for each halo

hal = halo.io.IO.read_catalogs('redshift', 0, simulation_directory, species='star')

In [ ]:
# all stellar properties have dictionary keys as 'star.*'
# list of star particle properties
for k in hal:
    if 'star.' in k:
        print(k)

In [ ]:
# find halos with star particles

hindices = np.where(hal['star.mass'] > 0)[0]

In [ ]:
# mass of all star particles in halo [M_sun]

hal['star.mass'][hindices]

In [ ]:
# number of star particles in halo [M_sun]

hal['star.number'][hindices]

In [ ]:
# radius that encloses 50%, 90% of the stellar mass [kpc physical]

print(hal['star.radius.50'][hindices])
print(hal['star.radius.90'][hindices])

In [ ]:
# derived property: stellar density (within R_50) as a derived property [M_sun / kpc^3]

hal.prop('star.density.50', hindices)

In [ ]:
# stellar velocity dispersion (standard deviation) at R_50 [km / s]

hal['star.vel.std.50'][hindices]

In [ ]:
# center-of-mass position and velocity of star particles [kpc comoving]

print(hal['star.position'][hindices])
print(hal['star.velocity'][hindices])

In [ ]:
# time (age of Universe) when galaxy formed 50%, 90%, etc of its current stellar mass [Gyr]

print(hal['star.form.time.50'][hindices])
print(hal['star.form.time.90'][hindices])
print(hal['star.form.time.95'][hindices])
print(hal['star.form.time.100'][hindices])

In [ ]:
# convert this to lookback-time via a derived property [Gyr]

hal.prop('star.form.time.lookback.50', hindices)

In [ ]:
# indices of member star particles
# for example, star particles assigned to halo 0 would be part_indices = hal['star.indies'][0]
# then if you read in the star particles in the snapshot file, you can access member star particles via 
# part['star'][property_name][part_indices]

hal['star.indices'][hindices[0]]

If you are reading halos at some snapshot at z > 0, and if the GizmoAnalysis package already generated baryonic particle pointers for tracking baryonic particles between any snapshot at z > 0 and the snapshot at z = 0, you additionally can store the pointer indices from member baryonic particles in each halo at z > 0 to the particle catalog at z = 0 by setting assign_species_pointers=True as follows.

There are no pointers at the snapshot at z = 0, becuase it would point to itself. 

In [ ]:
hal = halo.io.IO.read_catalogs('redshift', 1, simulation_directory, species='star', assign_species_pointers=True)

In [ ]:
hindices = np.where(hal['star.mass'] > 0)[0]

# indices of member star particles in the particle catalog at this snapshot
print(hal['star.indices'][hindices[3]])

# indices of member star particles in the particle catalog at the final snapshot at z = 0
# in other words, where these member star particles end up in the particle array at z = 0
print(hal['star.z0.indices'][hindices[3]])

# additional information stored in sub-dictionaries

In [ ]:
# dictionary of useful information about the simulation

hal.info

In [ ]:
# dictionary class of information about the cosmology of the simulation

hal.Cosmology

In [ ]:
# dictionary of information about this snapshot's index, scale-factor, redshift, time, lookback-time

hal.snapshot

In [ ]:
# dictionary of arrays about *all* snapshots of the simulation

print(hal.Snapshot.keys())
print(hal.Snapshot['redshift'])

# halo merger trees

The halo finder treats each snapshot as an independent catalog, so halos at different snapshots do not know about each other. ConsistentTrees produces halo merger trees that link halos over time.

ConsistentTrees applies some smoothing and physical consistency checks on halos over time, which leads to two important differences from the halo catalog: (1) not every halo in the catalog exists in the merger tree (especially those that are only marginally resolved) (2) some halos in the merger tree are 'phantom' halos that have been interpolated across snapshots but do not exist in the halo catalog at that snapshot.

As with the halo catalogs, the HaloAnalysis reader can check for files that contain baryonic (star or gas) particle properies at each snapshot, and it can read and append these baryonic properties to the halo merger trees. Unlike reading a halo catalog at a single snapshot, HaloAnalysis disables this feature by default when reading the merger trees, because it requires reading in about 600 hdf5 files and thus can be slow. To enable it, set species='star' in read_tree().

In [ ]:
# read halo merger trees across all snapshots
# this is a concatenated array of all halos across all snaphots, with pointers to progenitor and descendant halos

halt = halo.io.IO.read_tree(simulation_directory=simulation_directory)

In [ ]:
# read halo merger trees across all snapshots and read member star particle files at each snapshot
# this may take a while...

halt = halo.io.IO.read_tree(simulation_directory=simulation_directory, species='star')

In [ ]:
# if you only want to read star particle information at one or a few snapshots, you can specify which ones, which significantly speeds up the read time!
# for example, read star particles information only at snapshot 277 (z = 1) and 600 (z = 0)

halt = halo.io.IO.read_tree(simulation_directory=simulation_directory, species='star', species_snapshot_indices=[277, 600])

In [ ]:
# read halo merger trees across all snapshots and read member star particles and pointer files at select snapshots

halt = halo.io.IO.read_tree(simulation_directory=simulation_directory, species='star', species_snapshot_indices=[277, 600], assign_species_pointers=True)

In [ ]:
# halt is a dictionary of halo merger tree properties

for k in halt.keys():
    print(k)

In [ ]:
# each halo has its snapshot index (remember that the tree contains all halos at every snapshot)

halt['snapshot']

In [ ]:
# get all halos at snapshot 600 (z = 0) and print their masses

hindices = np.where(halt['snapshot'] == 600)[0]
print(halt['mass'][hindices])

In [ ]:
# get indices of member star particles at snapshot 277 and pointers tot their indices at z = 0 (snapshot 600)
hindices = np.where(halt['star.mass'] > 0)[0]
hindices = ut.array.get_indices(halt['snapshot'], 277, hindices)
hindex = hindices[10]

print(halt['star.indices'][hindex])
print(halt['star.z0.indices'][hindex])

In [ ]:
# alternately, get catalog of halos only at snapshot 600 from tree

hal = halo.io.IO.get_catalog_from_tree(halt, 600)

In [ ]:
# flag of whether halo is 'phantom' interpolation across snapshots

halt['am.phantom']

In [ ]:
# get the tree index of a halo's descendant at a later (usually the next) snapshot
# a negative value means that a halo does not have a descendant
# ConsistentTrees allows only one descendant per halo

print(halt['descendant.index'])

# for example, get descendant of halo index 100 and print its mass
hindex = 100
desc_index = halt['descendant.index'][hindex]
print(desc_index, halt['mass'][desc_index])

In [ ]:
# number of progenitor halos (can be arbitrarily large) at an earler (usually the previous) snapshot
# a negative value means that a halo does not have a progenitor

halt['progenitor.number']

In [ ]:
# tree index of main (most massive) progenitor
# loop over this to get list of main progenitors going back in time

halt['progenitor.main.index']

In [ ]:
# whether I am the main (most massive) progenitor of my descendant

halt['am.progenitor.main']

In [ ]:
# tree index of next co-progenitor

halt['progenitor.co.index']

In [ ]:
# tree index of my descendant at the final snapshot (z = 0)

halt['final.index']

In [ ]:
# example of walking the merger tree
# find all progenitors and list their masses

# start with a halo with tree index 0 (at z = 0)
hindex = 0
print(halt['snapshot'][hindex])
print(halt['mass'][hindex])

# find its progenitors, list their mass
prog_index = halt['progenitor.main.index'][hindex]
prog_indices = []
while prog_index >= 0:
    prog_indices.append(prog_index)
    prog_index = halt['progenitor.co.index'][prog_index]
print(prog_indices)
print(halt['mass'][prog_indices])

# for the main progenitor, find its progenitors, list their mass
hindex = prog_indices[0]
prog_index = halt['progenitor.main.index'][hindex]
prog_indices = []
while prog_index >= 0:
    prog_indices.append(prog_index)
    prog_index = halt['progenitor.co.index'][prog_index]
print(prog_indices)
print(halt['mass'][prog_indices])

In [ ]:
# example of walking the merger tree
# get list of main progenitors as far back as can go

hindex = 0
prog_main_index = hindex
prog_main_indices = []
while prog_main_index >= 0:
    prog_main_indices.append(prog_main_index)
    prog_main_index = halt['progenitor.main.index'][prog_main_index]

print(prog_main_indices)
print(halt['mass'][prog_main_indices])
print(halt['position'][prog_main_indices])

In [ ]:
# halt.prop() computes derived quantities and can make walking the halo merger tree much easier

# for halo with tree index = 0, get its main progenitors going back as far as can (including self)
hindex = 0
prog_indices = halt.prop('progenitor.main.indices', hindex)

# print mass history
print(prog_indices)
print(halt['mass'][prog_indices])

In [ ]:
# print stellar mass history - recall that not all snapshots necessarily have star particle information

print(halt['star.mass'][prog_indices])

In [ ]:
# you can do this for an array of halo indices as well
# it will return progenitor indices as 2-D array (halo number x progenitor number) with null values 
# (snapshot before a halo existed) as (safely) negative

hindices = np.where(halt['snapshot'] == 600)[0]
prog_indices = halt.prop('progenitor.main.indices', hindices)
print(prog_indices)

In [ ]:
# same for descendant halos (going forward in time as far as can)

hindices = np.where(halt['snapshot'] == 100)[0]
desc_indices = halt.prop('descendant.indices', hindices)
print(desc_indices)

In [ ]:
# alternately, get *all* of the progenitors at previous snapshot and print their masses

hindex = 2
prog_indices = halt.prop('progenitor.indices', hindex)

print(prog_indices)
print(halt['mass'][prog_indices])

In [ ]:
# this also works with an input list (array) of halos
# but now it returns a list of progenitor index arrays
# because each halo has a variable (and potentially unlimited) number of progenitors

hindices = np.where(halt['snapshot'] == 100)[0]
hindices = hindices[:10]
prog_indices = halt.prop('progenitor.indices', hindices)

print(prog_indices)

In [ ]:
# the halo catalogs contains history-based properties of each halo, such as peak mass or peak circular velocity
# you can use .prop() to compute these on the fly within the merger trees for any property
# you can get either the full history for that property or the peak (maximum) value
# again, you can do this for a single halo or an array of them

hindices = np.where(halt['snapshot'] == 600)[0]

# one halo
hindex = hindices[0]
print(halt.prop('mass.bound.history', hindex))
print(halt.prop('mass.bound.peak', hindex))

# array of halos
print(halt.prop('mass.bound.history', hindices))
print(halt.prop('mass.bound.peak', hindices))

# interfacting between halo catalog and merger trees

Each halo in the merger tree has a pointer to its array index in the halo catalog: 'catalog.index'

Conversely, each halo in the catalog has a pointer to its array index in the merger tree: 'tree.index'

You can use these to go back and forth between the two.

Note: halo merger tree ids/indices are unique across all snapshots, but halo catalog ids/indices are unique only at a given snapshot

In [ ]:
# read halo merger trees across all snapshots
halt = halo.io.IO.read_tree(simulation_directory=simulation_directory)

# read halo catalog at a snapshot
hal = halo.io.IO.read_catalogs('redshift', 2, simulation_directory=simulation_directory, species=None)

In [ ]:
# get index of the halo in the merger trees
cat_index = 100  # some halo of interest
print(hal['mass'][cat_index])
tree_index = hal['tree.index'][cat_index]
print(tree_index)
print(halt['mass'][tree_index])

In [ ]:
# conversely, starting from the halo merger tree, get index of the halo in the catalog
# because halo catalog indices are *not* unique across snapshots,
# you also need to use which snapshot index the halo points to

hindices = np.where(halt['snapshot'] == 172)[0]
tree_index = hindices[0]  # some halo of interest
print(halt['mass'][tree_index])
snapshot_index = halt['snapshot'][tree_index]  # index of snapshot in halo catalog
cat_index = halt['catalog.index'][tree_index]  # halo catalog index at snapshot
print(snapshot_index, cat_index)
print(hal['mass'][cat_index])

See halo.plot for examples of analyzing/plotting halos in the catalog and merger trees.

See utilities package for lower-level functions that may be useful.